In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from keras.models import Sequential
import tensorflow as tf
from keras.layers import *
from keras.optimizers import *

In [ ]:
# Set parameters here 
INPUT_SIZE = 256
mapping = {0:'normal', 1:'other'}
NUMCLASSES = len(mapping)
BATCH_SIZE = 16

folder = r'C:\Users\arung\OneDrive\Desktop\COVID 19 Chest Xray\nih'
SEED = 12345
np.random.seed(SEED)
tf.set_random_seed(SEED)

In [3]:
from skimage import exposure

def preprocess(img):
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    
    # Histogram Equalization
    #img_eq = exposure.equalize_hist(img)

    # Adaptive Equalization
    #img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    
    return img_rescale


In [4]:
train_datagen = ImageDataGenerator(
                                featurewise_center=False,
                                samplewise_center=False,
                                featurewise_std_normalization=False,
                                samplewise_std_normalization=False,
                                zca_whitening=False,
                                zca_epsilon=1e-06,
                                rotation_range=0,
                                width_shift_range=0.0,
                                height_shift_range=0.0,
                                brightness_range=None,
                                shear_range=0.0,
                                zoom_range=0.0,
                                channel_shift_range=0.0,
                                fill_mode="nearest",
                                cval=0.0,
                                horizontal_flip=False,
                                vertical_flip=False,
                                rescale=1./255,
                                preprocessing_function=preprocess,
                                data_format=None,
                                validation_split=0.2,
                                dtype=None,
                            )


In [5]:
df = pd.read_csv(folder + r'\allfiles.csv')
df = df[df.label!='pneumonia']
# This next line is to use only some x% of the data
df = df.sample(frac=1)
print(df['label'].value_counts())

normal    50500
other     35790
Name: label, dtype: int64


In [6]:
df.head()

,filename,label
30795,C:\Users\arung\OneDrive\Desktop\COVID 19 Chest...,normal
10022,C:\Users\arung\OneDrive\Desktop\COVID 19 Chest...,normal
46509,C:\Users\arung\OneDrive\Desktop\COVID 19 Chest...,normal
27011,C:\Users\arung\OneDrive\Desktop\COVID 19 Chest...,normal
30480,C:\Users\arung\OneDrive\Desktop\COVID 19 Chest...,normal


In [7]:


training_set = train_datagen.flow_from_dataframe(
                                            dataframe=df,
                                            directory='',
                                            x_col="filename",
                                            y_col="label",
                                            weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode="categorical",
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            seed=None,
                                            save_to_dir=None,
                                            save_prefix="",
                                            save_format="png",
                                            subset='training',
                                            interpolation="nearest",
                                            validate_filenames=True
                                        )

validation_set = train_datagen.flow_from_dataframe( dataframe=df,
                                            directory='',
                                            x_col="filename",
                                            y_col="label",
                                            weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode="categorical",
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            seed=None,
                                            save_to_dir=None,
                                            save_prefix="",
                                            save_format="png",
                                            subset='validation',
                                            interpolation="nearest",
                                            validate_filenames=True
                                            )

C:\Users\arung\anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 10116 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 60940 validated image filenames belonging to 2 classes.
Found 15234 validated image filenames belonging to 2 classes.


In [8]:
print(len(training_set)*BATCH_SIZE)

60944


In [9]:
def conv_block(ni, nf, size=3, stride=1):
    ni = 'ignored'
    return Sequential([
        Conv2D(nf, kernel_size=size, strides=stride, padding='same', bias=False), 
        BatchNormalization(),
        LeakyReLU(alpha=0.1)  
    ])

In [10]:
def triple_conv(ni, nf):
    return Sequential([
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    ])

In [11]:
def maxpooling():
    return MaxPool2D(2, strides=2)

In [12]:
model = Sequential()

model.add(InputLayer(input_shape=[INPUT_SIZE,INPUT_SIZE,3])) #keras will internally add batch dimension
model.add(conv_block(3, 8))
model.add(maxpooling())
model.add(conv_block(8, 16))
model.add(maxpooling())
model.add(triple_conv(16, 32))
model.add(maxpooling())
model.add(triple_conv(32, 64))
model.add(maxpooling())
model.add(triple_conv(64, 128))
model.add(maxpooling())
model.add(triple_conv(128, 256))
model.add(conv_block(256, 128, size=1))
model.add(conv_block(128, 256))
model.add(Conv2D(256, 2))
model.add(BatchNormalization()),
model.add(ReLU())
model.add(Flatten())
model.add(Dense(2,activation='softmax'))



model.compile(optimizer=Adam(lr=0.003), loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

C:\Users\arung\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, kernel_size=3, strides=1, padding="same", use_bias=False)`
  after removing the cwd from sys.path.


C:\Users\arung\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, kernel_size=3, strides=1, padding="same", use_bias=False)`
  after removing the cwd from sys.path.
C:\Users\arung\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=3, strides=1, padding="same", use_bias=False)`
  after removing the cwd from sys.path.
C:\Users\arung\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, kernel_size=1, strides=1, padding="same", use_bias=False)`
  after removing the cwd from sys.path.
C:\Users\arung\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=3, strides=1, padding="same", use_bias=False)`
  after removing the cwd from sys.path.
C:\Users\arung\anaconda3\lib\site-packages\ipyke

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 256, 256, 8)       248       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 8)       0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 128, 128, 16)      1216      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
sequential_7 (Sequential)    (None, 64, 64, 32)        10048     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
sequential_11 (Sequential)   (None, 32, 32, 64)       

In [13]:
#Training
#c_weights = {0: 0.4, 1: 0.6}

for epoch in range(0,50):
    print("Epoch",epoch)
    if epoch != 0:
        # Load Model Weights
        model.load_weights('model-normalized-fresh.h5')    
    history = model.fit_generator(training_set,
    steps_per_epoch=len(training_set),
                   epochs=1,
                   validation_data=validation_set,
                   validation_steps = len(validation_set))

    model.save_weights("model-normalized-fresh.h5")
    print("Saved model to disk after",epoch+1,"epochs.")


Epoch 0

Epoch 1/1


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node sequential_2/conv2d_1_1/convolution}}]]
	 [[metrics/accuracy/Identity/_579]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node sequential_2/conv2d_1_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
history.history.keys()

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.plot(history.history['loss'],color='red')
plt.plot(history.history['acc'],color='green')
plt.plot(history.history['val_loss'],color='magenta')
plt.plot(history.history['val_acc'],color='blue')
plt.show()

In [ ]:
df_test = pd.read_csv(folder + r'\alltestfiles.csv')
df_test = df_test[df_test.label!='pneumonia']
# This next line is to use only x% of the data
df_test = df_test.sample(n = df_test.shape[0]//20)
print(df_test['label'].value_counts())
print(df_test.shape)

In [ ]:

test_datagen = ImageDataGenerator(
                                featurewise_center=False,
                                samplewise_center=False,
                                featurewise_std_normalization=False,
                                samplewise_std_normalization=False,
                                zca_whitening=False,
                                zca_epsilon=1e-06,
                                rotation_range=0,
                                width_shift_range=0.0,
                                height_shift_range=0.0,
                                brightness_range=None,
                                shear_range=0.0,
                                zoom_range=0.0,
                                channel_shift_range=0.0,
                                fill_mode="nearest",
                                cval=0.0,
                                horizontal_flip=False,
                                vertical_flip=False,
                                rescale=None,
                                preprocessing_function=preprocess,
                                data_format=None,
                                validation_split=0.0,
                                dtype=None,
                            )

test_set = test_datagen.flow_from_dataframe(
                                            dataframe=df_test,
                                            directory='',
                                            x_col="filename",
                                            y_col="label",
                                            weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode=None,
                                            batch_size=1,
                                            shuffle=False,
                                            seed=None,
                                            save_to_dir=folder + '\\test',
                                            save_prefix="",
                                            save_format="png",
                                            subset='training',
                                            interpolation="nearest",
                                            validate_filenames=True
                                        )


In [ ]:
pred=model.predict_generator(test_set,verbose=1,steps=len(test_set)/1 )



In [ ]:
(len(pred))

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

predicted_class_indices.shape

In [ ]:
conf = np.zeros([NUMCLASSES,NUMCLASSES],'float32')
count = 0
for row in df_test['label']:
    if row=='normal':
        realclass = 0
    else:
        realclass = 1
    try:
        conf[realclass,predicted_class_indices[count]] += 1
    except:
        break
    count += 1
    
conf[0,:] /= 496
conf[1,:] /= 779

In [ ]:
conf